# Machine learning for text classification
with scikit-learn and nltk


## Agenda

- Model building in scikit-learn
- Representing text as numerical data
- Reading a text-based dataset into pandas
- Vectorizing our dataset
- Building and evaluating a model
- Comparing models
- Examining a model for further insight
- Tuning the vectorizer (discussion)
- Some NLP tools to preprocess text

In [4]:
# for Python 2: use print only as a function
from __future__ import print_function

## Model building in scikit-learn

In [5]:
# load the iris dataset as an example
from sklearn.datasets import load_iris
iris = load_iris()

In [3]:
# store the feature matrix (X) and label vector (y)
X = iris.data
y = iris.target

In [4]:
# check the shapes of X and y
print(X.shape)
print(y.shape)
print(iris.feature_names)
print(X[0])
print(y[0])

(150, 4)
(150,)
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
[5.1 3.5 1.4 0.2]
0


In [5]:
# examine the first 5 rows of the feature matrix (including the feature names)
# to install pandas, 'pip install pandas' or 'conda install pandas'
import pandas as pd
pd.DataFrame(X, columns=iris.feature_names).head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [6]:
# examine the label vector
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In order to **build a model**, the features must be **numeric**, and every observation must have the **same features in the same order**.

In [7]:
# import the class
from sklearn.neighbors import KNeighborsClassifier

# instantiate the model (with the default parameters)
knn = KNeighborsClassifier()

# fit the model with data (occurs in-place)
knn.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In order to **make a prediction**, the new observation must have the **same features as the training observations**, both in number and meaning.

In [8]:
# predict the response for a new observation
knn.predict([[3.2, 4.9, 4, 2.1]])

array([1])

Scikit-learn classical workflow:
1. import
2. instantiate
3. fit
4. predict

## Representing text as numerical data

In [9]:
# example text for model training (SMS messages)
simple_train = ["call you tonight", "Call me a cab", "please call me... PLEASE!"]

From the [scikit-learn documentation](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction):

> Text Analysis is a major application field for machine learning algorithms. However the raw data, a sequence of symbols cannot be fed directly to the algorithms themselves as most of them expect **numerical feature vectors with a fixed size** rather than the **raw text documents with variable length**.

We will use [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) to "convert text into a matrix of token counts":

In [10]:
# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [11]:
# learn the "vocabulary" of the training data (occurs in-place)
vect.fit(simple_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [12]:
# examine the fitted vocabulary
vect.get_feature_names()

['cab', 'call', 'me', 'please', 'tonight', 'you']

In [13]:
# transform training data into a "document-term matrix'
simple_train_dtm = vect.transform(simple_train)
simple_train_dtm

<3x6 sparse matrix of type '<class 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [14]:
# convert sparse matrix to a dense matrix
simple_train_dtm.toarray()

array([[0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 0],
       [0, 1, 1, 2, 0, 0]])

In [15]:
# examine the vocabulary and document-term matrix together
pd.DataFrame(simple_train_dtm.toarray(), columns=vect.get_feature_names())

,cab,call,me,please,tonight,you
0,0,1,0,0,1,1
1,1,1,1,0,0,0
2,0,1,1,2,0,0


From the [scikit-learn documentation](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction):

> In this scheme, features and samples are defined as follows:

> - Each individual token occurrence frequency (normalized or not) is treated as a **feature**.
> - The vector of all the token frequencies for a given document is considered a multivariate **sample**.

> A **corpus of documents** can thus be represented by a matrix with **one row per document** and **one column per token** (e.g. word) occurring in the corpus.

> We call **vectorization** the general process of turning a collection of text documents into numerical feature vectors. This specific strategy (tokenization, counting and normalization) is called the **Bag of Words** or _"Bag of n-grams"_ representation. Documents are described by word occurrences while completely ignoring the relative position information of the words in the document.

In [16]:
# check the type of the document-term matrix
type(simple_train_dtm)

scipy.sparse.csr.csr_matrix

In [17]:
# examine the sparse matrix contents
print(simple_train_dtm)

  (0, 1)	1
  (0, 4)	1
  (0, 5)	1
  (1, 0)	1
  (1, 1)	1
  (1, 2)	1
  (2, 1)	1
  (2, 2)	1
  (2, 3)	2


From the [scikit-learn documentation](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction):

> As most documents will typically use a very small subset of the words used in the corpus, the resulting matrix will have **many feature values that are zeros** (typically more than 99% of them).

> For instance, a collection of 10,000 short text documents (such as emails) will use a vocabulary with a size in the order of 100,000 unique words in total while each document will use 100 to 1000 unique words individually.

> In order to be able to **store such a matrix in memory** but also to **speed up operations**, implementations will typically use a **sparse representation** such as the implementations available in the `scipy.sparse` package.

In [18]:
# example text for model testing
simple_test = ["please don't call me"]

In order to **make a prediction**, the new observation must have the **same features as the training observations**, both in number and meaning.

In [19]:
# transform testing data into a document-term matrix (using existing vocabulary)
simple_test_dtm = vect.transform(simple_test)
simple_test_dtm.toarray()

array([[0, 1, 1, 1, 0, 0]])

In [20]:
# examine the vocabulary and document-term matrix together
pd.DataFrame(simple_test_dtm.toarray(), columns=vect.get_feature_names())

,cab,call,me,please,tonight,you
0,0,1,1,1,0,0


**Summary:**

- `vect.fit(train)` **learns the vocabulary** of the training data
- `vect.transform(train)` uses the **fitted vocabulary** to build a document-term matrix from the training data
- `vect.transform(test)` uses the **fitted vocabulary** to build a document-term matrix from the testing data (and **ignores tokens** it hasn't seen before)

# Example: learning to predict SMS spam
## Reading tsv dataset into pandas

In [21]:
# read file into pandas using a relative path
path = "data/sms.tsv"
sms = pd.read_table(path, header=None, names=["label", "message"])

In [22]:
# alternative: read file into pandas from a URL
#url = "http://people.rennes.inria.fr/Francois.Coste/pub/sms.tsv"
#sms = pd.read_table(url, header=None, names=['label', "message"])

## Inspect raw data

In [23]:
# examine the shape
sms.shape

(5572, 2)

In [24]:
# examine the first 10 rows
sms.head(10)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [25]:
# examine the class distribution
sms.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [26]:
# many other things could be done here....

## Split data into train and test sets

In [27]:
# Extract target labels
X = sms.message # equivalent to X = sms["message"]
y = sms.label.map({"ham":0, "spam":1}) # transformation to [0,1] labels
print(X.shape)
print(y.shape)

(5572,)
(5572,)


In [28]:
# Use train_test_split of sklearn.model_selection to initialize X_train, X_test, y_train, y_test (with default values and random_state=1)

# TODO: your code here
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [29]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4179,)
(1393,)
(4179,)
(1393,)


Did you expect to obtain these values?

## Vectorize sentences used for _training_

In [30]:
# Compute X_train_dtm, the document-term matrix of sentences in the training set

# TODO: your code here

# instantiate the vectorizer
vect = CountVectorizer()

# learn training data vocabulary, then use it to create a document-term matrix
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
# or equivalently: combine fit and transform into a single step
X_train_dtm = vect.fit_transform(X_train)

In [31]:
# Display informations on X_train_dtm result
X_train_dtm

<4179x7456 sparse matrix of type '<class 'numpy.int64'>'
	with 55209 stored elements in Compressed Sparse Row format>

In [32]:
# have a look at the vectorized training sentences
pd.DataFrame(X_train_dtm.toarray(), columns=vect.get_feature_names())

,00,000,008704050406,0121,01223585236,01223585334,0125698789,02,0207,02072069400,...,zed,zeros,zhong,zindgi,zoe,zoom,zouk,zyada,Ã¨n,ã€¨ud
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# It is great to have a sparse representation...
print(X_train_dtm)

  (0, 2022)	1
  (0, 4779)	1
  (0, 4662)	1
  (0, 6892)	1
  (0, 6656)	1
  (0, 50)	1
  (0, 4743)	1
  (0, 4375)	1
  (0, 1552)	1
  (0, 264)	1
  (0, 4983)	1
  (0, 7424)	1
  (0, 3170)	1
  (0, 2864)	2
  (0, 4987)	1
  (0, 1572)	1
  (0, 3880)	1
  (0, 5479)	1
  (0, 3971)	1
  (0, 4781)	1
  (0, 5193)	1
  (0, 3181)	1
  (0, 509)	1
  (1, 6758)	1
  (1, 3316)	1
  :	:
  (4177, 3700)	1
  (4177, 837)	1
  (4177, 307)	1
  (4177, 6662)	1
  (4177, 6034)	1
  (4177, 4508)	1
  (4177, 2556)	1
  (4177, 5490)	1
  (4177, 254)	1
  (4177, 2744)	1
  (4177, 4778)	1
  (4177, 4446)	1
  (4177, 4255)	1
  (4177, 3629)	1
  (4177, 7257)	1
  (4177, 1574)	1
  (4177, 6887)	1
  (4177, 3738)	1
  (4177, 5656)	1
  (4177, 6514)	1
  (4177, 6656)	1
  (4178, 5999)	1
  (4178, 7257)	1
  (4178, 4238)	1
  (4178, 1691)	1


TODO: In the code above, replace calls to ```fit``` and ```transform``` functions by one call to ```fit_transform```

## Vectorize sentences that will be used for _testing_

In [34]:
# Build X_test_dtm

# TODO: your code here

# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

<1393x7456 sparse matrix of type '<class 'numpy.int64'>'
	with 17604 stored elements in Compressed Sparse Row format>

TODO: Comment the difference with vectorization of training set

## Building and evaluating a model

We will use [multinomial Naive Bayes](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html):

> The multinomial Naive Bayes classifier is suitable for classification with **discrete features** (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work.

In [35]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [36]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

CPU times: user 4.88 ms, sys: 826 Âµs, total: 5.7 ms
Wall time: 38.4 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [37]:
# make class predictions for test sentences (using X_test_dtm representation)
y_pred_class = nb.predict(X_test_dtm)
y_pred_class

array([0, 0, 0, ..., 0, 1, 0])

In [38]:
# tranformation into a boolean array
print(y_pred_class==1)

[False False False ... False  True False]


In [39]:
# some sms predicted as spam
print(X_test[y_pred_class==1].head())

147     FreeMsg Why haven't you replied to my text? I'...
4517    Congrats! 2 mobile 3G Videophones R yours. cal...
574                                Waiting for your call.
3316    FREE MESSAGE Activate your 500 FREE Text Messa...
1929    Call from 08702490080 - tells u 2 call 0906635...
Name: message, dtype: object


In [40]:
# compute accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.9885139985642498

In [41]:
# compute the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

array([[1203,    5],
       [  11,  174]])

In [42]:
## pandas' indexing by boolean array
#val_df = pd.DataFrame(['A','B','C','D','E'])
#bool_array = [True, True, False, True, False]
#val_df[bool_array]

In [43]:
# print message text for the false positives (ham incorrectly classified as spam)
X_test[(y_pred_class==1) & (y_test==0)] # using pandas' indexing by boolean array

574               Waiting for your call.
3375             Also andros ice etc etc
45      No calls..messages..missed calls
3415             No pic. Please re-send.
1988    No calls..messages..missed calls
Name: message, dtype: object

In [44]:
# print message text for the false negatives (spam incorrectly classified as ham)
X_test[(y_pred_class < y_test)]

3132    LookAtMe!: Thanks for your purchase of a video...
5       FreeMsg Hey there darling it's been 3 week's n...
3530    Xmas & New Years Eve tickets are now on sale f...
684     Hi I'm sue. I am 20 years old and work as a la...
1875    Would you like to see my XXX pics they are so ...
1893    CALL 09090900040 & LISTEN TO EXTREME DIRTY LIV...
4298    thesmszone.com lets you send free anonymous an...
4949    Hi this is Amy, we will be sending you a free ...
2821    INTERFLORA - Â“It's not too late to order Inter...
2247    Hi ya babe x u 4goten bout me?' scammers getti...
4514    Money i have won wining number 946 wot do i do...
Name: message, dtype: object

In [45]:
# example false negative
X_test[3132]

"LookAtMe!: Thanks for your purchase of a video clip from LookAtMe!, you've been charged 35p. Think you can do better? Why not send a video in a MMSto 32323."

### Using probability scores

**predict_proba(X)**

    Return probability estimates for the test vector X.
    Parameters:	
    X : array-like, shape = [n_samples, n_features]
    Returns: 
    C : array-like, shape = [n_samples, n_classes]

        Returns the probability of the samples for each class in the model. The columns correspond to the classes in sorted order, as they appear in the attribute classes.


In [46]:
X_test_dtm

<1393x7456 sparse matrix of type '<class 'numpy.int64'>'
	with 17604 stored elements in Compressed Sparse Row format>

In [47]:
nb.predict_proba(X_test_dtm)

array([[9.97122551e-01, 2.87744864e-03],
       [9.99981651e-01, 1.83488846e-05],
       [9.97926987e-01, 2.07301295e-03],
       ...,
       [9.99998910e-01, 1.09026171e-06],
       [1.86697467e-10, 1.00000000e+00],
       [9.99999996e-01, 3.98279868e-09]])

In [48]:
# calculate predicted probabilities for X_test_dtm (poorly calibrated)
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([2.87744864e-03, 1.83488846e-05, 2.07301295e-03, ...,
       1.09026171e-06, 1.00000000e+00, 3.98279868e-09])

In [49]:
# calculate AUC
auc = metrics.roc_auc_score(y_test, y_pred_prob)
auc

0.9866431000536962

In [2]:
import numpy as np
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob, pos_label=1)

NameError: name 'y_test' is not defined

In [1]:
import matplotlib.pyplot as plt
plt.plot(fpr,tpr,label=f"Naive Bayes, auc={auc:.5}")
plt.legend(loc=4)
plt.show()

NameError: name 'fpr' is not defined

## Comparing models

We will compare multinomial Naive Bayes with [logistic regression](http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression):

> Logistic regression, despite its name, is a **linear model for classification** rather than regression. Logistic regression is also known in the literature as logit regression, maximum-entropy classification (MaxEnt) or the log-linear classifier. In this model, the probabilities describing the possible outcomes of a single trial are modeled using a logistic function.

In [52]:
# Learn and evaluate a logistic regression classifier on the same data set
# (using LogisticRegression from sklearn.linear_model) 

# TODO: your code here

# import and instantiate a logistic regression model 
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='lbfgs')

In [53]:
# train the model using X_train_dtm
%time logreg.fit(X_train_dtm, y_train)

CPU times: user 107 ms, sys: 5.9 ms, total: 113 ms
Wall time: 66.3 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [54]:
# make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test_dtm)

In [55]:
# calculate predicted probabilities for X_test_dtm (well calibrated)
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([0.00959377, 0.00295662, 0.00452424, ..., 0.031302  , 0.99748962,
       0.00119521])

In [56]:
# calculate accuracy
metrics.accuracy_score(y_test, y_pred_class)

0.9877961234745154

In [57]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

0.9936280651512441

## Examining Naive Bayes model for further insight 
We will examine the our **trained Naive Bayes model** to calculate the approximate **"spamminess" of each token**.

In [58]:
# store the vocabulary of X_train
X_train_tokens = vect.get_feature_names()
len(X_train_tokens)

7456

In [59]:
# examine the first 50 tokens
print(X_train_tokens[0:50])

['00', '000', '008704050406', '0121', '01223585236', '01223585334', '0125698789', '02', '0207', '02072069400', '02073162414', '02085076972', '021', '03', '04', '0430', '05', '050703', '0578', '06', '07', '07008009200', '07090201529', '07090298926', '07123456789', '07732584351', '07734396839', '07742676969', '0776xxxxxxx', '07781482378', '07786200117', '078', '07801543489', '07808', '07808247860', '07808726822', '07815296484', '07821230901', '07880867867', '0789xxxxxxx', '07946746291', '0796xxxxxx', '07973788240', '07xxxxxxxxx', '08', '0800', '08000407165', '08000776320', '08000839402', '08000930705']


In [60]:
# examine the last 50 tokens
print(X_train_tokens[-50:])

['yer', 'yes', 'yest', 'yesterday', 'yet', 'yetunde', 'yijue', 'ym', 'ymca', 'yo', 'yoga', 'yogasana', 'yor', 'yorge', 'you', 'youdoing', 'youi', 'youphone', 'your', 'youre', 'yourjob', 'yours', 'yourself', 'youwanna', 'yowifes', 'yoyyooo', 'yr', 'yrs', 'ything', 'yummmm', 'yummy', 'yun', 'yunny', 'yuo', 'yuou', 'yup', 'zac', 'zaher', 'zealand', 'zebra', 'zed', 'zeros', 'zhong', 'zindgi', 'zoe', 'zoom', 'zouk', 'zyada', 'Ã¨n', 'ã€¨ud']


In [61]:
# Naive Bayes counts the number of times each token appears in each class
# trailing underscore is scikit convention for attributes that are learned during model fitting
nb.feature_count_

array([[ 0.,  0.,  0., ...,  1.,  1.,  1.],
       [ 5., 23.,  2., ...,  0.,  0.,  0.]])

In [62]:
# rows represent classes, columns represent tokens
nb.feature_count_.shape

(2, 7456)

In [63]:
# number of times each token appears across all HAM messages
ham_token_count = nb.feature_count_[0, :]
ham_token_count

array([0., 0., 0., ..., 1., 1., 1.])

In [64]:
# number of times each token appears across all SPAM messages
spam_token_count = nb.feature_count_[1, :]
spam_token_count

array([ 5., 23.,  2., ...,  0.,  0.,  0.])

In [65]:
# create a DataFrame of tokens with their separate ham and spam counts
tokens = pd.DataFrame({"token":X_train_tokens, "ham":ham_token_count, "spam":spam_token_count}).set_index("token")
tokens.head()

,ham,spam
token,,
00,0.0,5.0
000,0.0,23.0
008704050406,0.0,2.0
0121,0.0,1.0
01223585236,0.0,1.0


In [66]:
# examine 5 random DataFrame rows
tokens.sample(5, random_state=6)

,ham,spam
token,,
very,64.0,2.0
nasty,1.0,1.0
villa,0.0,1.0
beloved,1.0,0.0
textoperator,0.0,2.0


In [67]:
# Naive Bayes counts the number of observations in each class
nb.class_count_

array([3617.,  562.])

Before we can calculate the _"spamminess"_ of each token, we need to avoid **multiplying by zero** and account for the **class imbalance**.

In [68]:
# add 1 to ham and spam counts to avoid 0 probabilities
tokens['ham'] = tokens['ham'] + 1
tokens['spam'] = tokens['spam'] + 1
tokens.sample(5, random_state=6)

,ham,spam
token,,
very,65.0,3.0
nasty,2.0,2.0
villa,1.0,2.0
beloved,2.0,1.0
textoperator,1.0,3.0


In [69]:
# convert the ham and spam counts into frequencies
tokens['ham'] = tokens['ham'] / nb.class_count_[0]
tokens['spam'] = tokens['spam'] / nb.class_count_[1]
tokens.sample(5, random_state=6)

,ham,spam
token,,
very,0.017971,0.005338
nasty,0.000553,0.003559
villa,0.000276,0.003559
beloved,0.000553,0.001779
textoperator,0.000276,0.005338


In [70]:
# calculate the ratio of spam-to-ham for each token
tokens['spam_ratio'] = tokens['spam'] / tokens['ham']
tokens.sample(5, random_state=6)

,ham,spam,spam_ratio
token,,,
very,0.017971,0.005338,0.297044
nasty,0.000553,0.003559,6.435943
villa,0.000276,0.003559,12.871886
beloved,0.000553,0.001779,3.217972
textoperator,0.000276,0.005338,19.307829


In [71]:
# examine the DataFrame sorted by spam_ratio
# note: use sort() instead of sort_values() for pandas 0.16.2 and earlier
tokens.sort_values('spam_ratio', ascending=False)

,ham,spam,spam_ratio
token,,,
claim,0.000276,0.158363,572.798932
prize,0.000276,0.135231,489.131673
150p,0.000276,0.087189,315.361210
tone,0.000276,0.085409,308.925267
guaranteed,0.000276,0.076512,276.745552
18,0.000276,0.069395,251.001779
cs,0.000276,0.065836,238.129893
www,0.000553,0.129893,234.911922
1000,0.000276,0.056940,205.950178


In [72]:
# look up the spam_ratio for a given token
tokens.loc["dating", "spam_ratio"]

83.66725978647686

## Tuning the vectorizer (discussion)

Thus far, we have been using the default parameters of [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html):

In [73]:
# show default parameters for CountVectorizer
vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

However, the vectorizer is worth tuning, just like a model is worth tuning! Here are a few parameters that you might want to tune:

- **stop_words:** string {'english'}, list, or None (default)
    - If 'english', a built-in stop word list for English is used.
    - If a list, that list is assumed to contain stop words, all of which will be removed from the resulting tokens.
    - If None, no stop words will be used.

In [74]:
# remove English stop words
vect_sw = CountVectorizer(stop_words='english')

In [75]:
vect_sw.fit(X_train)
X_train_dtm_sw = vect_sw.transform(X_train)
print(X_train_dtm_sw.shape) 
print(X_train_dtm.shape)

(4179, 7204)
(4179, 7456)


- **ngram_range:** tuple (min_n, max_n), default=(1, 1)
    - The lower and upper boundary of the range of n-values for different n-grams to be extracted.
    - All values of n such that min_n <= n <= max_n will be used.

In [76]:
# include 1-grams and 2-grams
vect_ngram = CountVectorizer(ngram_range=(1, 2))

- **max_df:** float in range [0.0, 1.0] or int, default=1.0
    - When building the vocabulary, ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words).
    - If float, the parameter represents a proportion of documents.
    - If integer, the parameter represents an absolute count.

In [77]:
# ignore terms that appear in more than 50% of the documents
vect_maxdf = CountVectorizer(max_df=0.5)

- **min_df:** float in range [0.0, 1.0] or int, default=1
    - When building the vocabulary, ignore terms that have a document frequency strictly lower than the given threshold. (This value is also called _"cut-off"_ in the literature.)
    - If float, the parameter represents a proportion of documents.
    - If integer, the parameter represents an absolute count.

In [78]:
# only keep terms that appear in at least 2 documents
vect_mindf = CountVectorizer(min_df=2)

**Guidelines for tuning CountVectorizer:**

- Use your knowledge of the **problem** and the **text**, and your understanding of the **tuning parameters**, to help you decide what parameters to tune and how to tune them.
- **Experiment**, and let the data tell you the best approach!

### TF-IDF weighting

From the [scikit-learn documentation](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html):

> Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.

> To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called tf for Term Frequencies.

> Another refinement on top of tf is to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.

> This downscaling is called tfâ€“idf for â€œTerm Frequency times Inverse Document Frequencyâ€.

In [79]:
# Computing TF only
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False)
X_train_tf = tf_transformer.fit_transform(X_train_dtm)
X_train_tf.shape

(4179, 7456)

In [80]:
### Computing TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_dtm)
X_train_tfidf.shape

(4179, 7456)

In [81]:
from sklearn.naive_bayes import MultinomialNB
nb_tfidf = MultinomialNB().fit(X_train_tfidf, y_train)

In [82]:
X_test_tfidf = tfidf_transformer.transform(X_test_dtm)
print(X_test_tfidf.shape)
y_pred_class_tfidf = nb_tfidf.predict(X_test_tfidf)

(1393, 7456)


In [83]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class_tfidf)

0.964824120603015

In [84]:
metrics.confusion_matrix(y_test, y_pred_class_tfidf)

array([[1208,    0],
       [  49,  136]])

### Building a pipeline

> In order to make the vectorizer => transformer => classifier easier to work with, scikit-learn provides a Pipeline class that behaves like a compound classifier:

In [85]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('nb', MultinomialNB()),
                    ])

> The names vect, tfidf and clf (classifier) are arbitrary. We shall see their use in the section on grid search, below. We can now train the model with a single command:

In [86]:
text_clf.fit(X_train, y_train)  

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...linear_tf=False, use_idf=True)), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [87]:
pred = text_clf.predict(X_test)

In [88]:
metrics.confusion_matrix(y_test, pred)

array([[1208,    0],
       [  49,  136]])

In [89]:
print(metrics.classification_report(y_test, pred, target_names=["ham","spam"]))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1208
        spam       1.00      0.74      0.85       185

   micro avg       0.96      0.96      0.96      1393
   macro avg       0.98      0.87      0.91      1393
weighted avg       0.97      0.96      0.96      1393



__How does pipeline chain the estimators?__   
By successive 'fit' and 'transform' (except the last estimator that is only fitted)   
From [scikit-learn documentation](http://scikit-learn.org/stable/modules/pipeline.html):
> Calling fit on the pipeline is the same as calling fit on each estimator in turn, transform the input and pass it on to the next step. The pipeline has all the methods that the last estimator in the pipeline has, i.e. if the last estimator is a classifier, the Pipeline can be used as a classifier. If the last estimator is a transformer, again, so is the pipeline.

### Parameter tuning using grid search

Parameters of the estimators in the pipeline can be accessed using the < estimator \> \_\_ < parameter \> syntax:

In [90]:
text_clf.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=None, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
         tokenizer=None, vocabulary=None),
 'tfidf': T

In [91]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'nb__alpha': (1e-2, 1e-3),
}
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1, cv=3, return_train_score=True)

In [92]:
gs_clf = gs_clf.fit(X_train, y_train)

The result of calling fit on a GridSearchCV object is a classifier that we can use to predict:

In [93]:
gs_clf.predict(X_test)

array([0, 0, 0, ..., 0, 1, 0])

In [94]:
gs_clf.best_score_  

0.9858817899018905

In [95]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))


nb__alpha: 0.01
tfidf__use_idf: True
vect__ngram_range: (1, 2)


In [96]:
gs_clf.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('nb', MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True))])

In [97]:
gs_clf.best_estimator_.get_params()["nb"]

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [98]:
gs_clf.cv_results_

{'mean_fit_time': array([0.12735987, 0.23823667, 0.08943391, 0.23742183, 0.08908391,
        0.25156323, 0.10549577, 0.23110127]),
 'std_fit_time': array([0.01454882, 0.01888441, 0.00262615, 0.00319044, 0.00065503,
        0.01272735, 0.02231945, 0.01386171]),
 'mean_score_time': array([0.04013276, 0.07230846, 0.03761133, 0.06880434, 0.03787867,
        0.070834  , 0.03723383, 0.04345743]),
 'std_score_time': array([0.00324054, 0.00153086, 0.00110409, 0.00045262, 0.00046665,
        0.00083656, 0.000408  , 0.00719962]),
 'param_nb__alpha': masked_array(data=[0.01, 0.01, 0.01, 0.01, 0.001, 0.001, 0.001, 0.001],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_tfidf__use_idf': masked_array(data=[True, True, False, False, True, True, False, False],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_vect__ngram_range':

In [99]:
import pandas as pd
df = pd.DataFrame(gs_clf.cv_results_)
print(df)

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.127360      0.014549         0.040133        0.003241   
1       0.238237      0.018884         0.072308        0.001531   
2       0.089434      0.002626         0.037611        0.001104   
3       0.237422      0.003190         0.068804        0.000453   
4       0.089084      0.000655         0.037879        0.000467   
5       0.251563      0.012727         0.070834        0.000837   
6       0.105496      0.022319         0.037234        0.000408   
7       0.231101      0.013862         0.043457        0.007200   

  param_nb__alpha param_tfidf__use_idf param_vect__ngram_range  \
0            0.01                 True                  (1, 1)   
1            0.01                 True                  (1, 2)   
2            0.01                False                  (1, 1)   
3            0.01                False                  (1, 2)   
4           0.001                 True                  (1, 1)   


# Some NLP tools to preprocess text
See also:
- [Speech and Language Processing](https://web.stanford.edu/~jurafsky/slp3/) book by Dan Jurafsky and James H. Martin
- [Natural Language Processing with Python](https://www.nltk.org/book/) book by Steven Bird, Ewan Klein, and Edward Loper
- [Text Mining Online](https://textminingonline.com/) tutorials

In [100]:
# download ressources
import nltk

# download all popular ressources
#nltk.download('popular')

# download required ressources for this notebook
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading stopwords: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>
[nltk_data] Error loading punkt: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>
[nltk_data] Error loading wordnet: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno -2] Name or service not known>


False

### Stop words

In [101]:
from nltk.corpus import stopwords
set(stopwords.words('english'))

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

### Tokenizing words and sentences
_Sentence tokenizer_ breaks text paragraph into sentences.   
_Word tokenizer_ breaks text paragraph into words.

In [102]:
from nltk.tokenize import sent_tokenize, word_tokenize

example_text = "Hello Mr. Smith, how are you doing today? The weather is great, and Python is awesome. The sky is pinkish-blue. You shouldn't eat cardboard."

print("Token sentences:\n",sent_tokenize(example_text))
print("Token words:\n",word_tokenize(example_text))

Token sentences:
 ['Hello Mr. Smith, how are you doing today?', 'The weather is great, and Python is awesome.', 'The sky is pinkish-blue.', "You shouldn't eat cardboard."]
Token words:
 ['Hello', 'Mr.', 'Smith', ',', 'how', 'are', 'you', 'doing', 'today', '?', 'The', 'weather', 'is', 'great', ',', 'and', 'Python', 'is', 'awesome', '.', 'The', 'sky', 'is', 'pinkish-blue', '.', 'You', 'should', "n't", 'eat', 'cardboard', '.']


```
# Unsupervised learning of tokenizer: PunktSentenceTokenizer
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
nltk.download("state_union")
train_text = state_union.raw("2005-GWBush.txt")
test_text = state_union.raw("2006-GWBush.txt")
custom_sent_tokenizer = PunktSentenceTokenizer(train_text)
tokenized = custom_sent_tokenizer.tokenize(test_text)
```

## Lexicon Normalization
### Stemming ("racinisation")
Stemming is the process for reducing inflected (or sometimes derived) words to their stem, base or root formâ€”generally a written word form. 

In [103]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

example_words = ["car", "cars", "churches", "going", "gone", "goes", "went", "geese"]
print("words:", example_words)

ps = PorterStemmer()
print("stems:",[ps.stem(w) for w in example_words])

words: ['car', 'cars', 'churches', 'going', 'gone', 'goes', 'went', 'geese']
stems: ['car', 'car', 'church', 'go', 'gone', 'goe', 'went', 'gees']


In [104]:
new_text = "It is important to by very pythonly while you are pythoning with python. All pythoners have pythoned poorly at least once."

In [105]:
words = word_tokenize(new_text)
print([ps.stem(w) for w in words])

['It', 'is', 'import', 'to', 'by', 'veri', 'pythonli', 'while', 'you', 'are', 'python', 'with', 'python', '.', 'all', 'python', 'have', 'python', 'poorli', 'at', 'least', 'onc', '.']


#### how to add stemming support to CountVectorizer (sklearn)

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk.stem

stemmer = nltk.stem.SnowballStemmer('english')
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

vectorizer_s = StemmedCountVectorizer(min_df=3, analyzer="word", stop_words='english')

### Lemmatizing
Lemmatization reduces words to their base word, which is linguistically correct lemmas (canonical form for unit of meaning).

In [2]:
from nltk.stem import WordNetLemmatizer
#nltk.download("wordnet")
print("Words:")
print([w for w in example_words])

lemmatizer = WordNetLemmatizer()
print("Lemmatized as noun (default):")
print([lemmatizer.lemmatize(w) for w in example_words])
    
print("Lemmatized as verb:")
print([lemmatizer.lemmatize(w,'v') for w in example_words])

 

Words:


NameError: name 'example_words' is not defined

### POS (Part of speech) tagging

In [3]:
# We need to tokenize first the text
text = word_tokenize("They refuse to permit us to obtain the refuse permit")
print(text,"\n")
# Then we can perform POS
print(nltk.pos_tag(text))

NameError: name 'word_tokenize' is not defined





[Penn Treebank POS tags](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html):

CC:	coordinating conjunction   
CD:	cardinal digit   
DT:	determiner   
EX:	existential there (like: _"there is"_ ... think of it like _"there exists"_)   
FW:	foreign word   
IN:	preposition/subordinating conjunction   
JJ:	adjective	_"big"_   
JJR:	adjective, comparative _"bigger"_   
JJS:	adjective, superlative _"biggest"_   
LS:	list marker	1)   
MD:	modal	could, will   
NN:	noun, singular _"desk"_   
NNS:	noun plural	_"desks"_   
NNP:	proper noun, singular _"Harrison"_   
NNPS:	proper noun, plural	_"Americans"_   
PDT:	predeterminer _"all the kids"_   
POS:	possessive ending _"parent's"_   
PRP:	personal pronoun	_"I, he, she"_   
PRP\$:	possessive pronoun	_"my, his, hers"_   
RB:	adverb	_"very, silently,"_   
RBR:	adverb, comparative	_"better"_   
RBS:	adverb, superlative	_"best"_   
RP:	particle	_"give up"_   
TO:	to	_"go 'to' the store."_   
UH:	interjection	_"errrrrrrrm"_   
VB:	verb, base form	_"take"_   
VBD:	verb, past tense	_"took"_   
VBG:	verb, gerund/present participle	_"taking"_   
VBN:	verb, past participle	_"taken"_   
VBP:	verb, sing. present, non-3d	_"take"_   
VBZ:	verb, 3rd person sing. present	_"takes"_   
WDT:	wh-determiner	_"which"_   
WP:	wh-pronoun	_"who, what"_   
WP\$:	possessive wh-pronoun	_"whose"_   
WRB:	wh-abverb	_"where, when"_   

### A more complete example

In [109]:
from nltk.corpus import state_union
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download("state_union")
su_text = state_union.raw("2006-GWBush.txt")
sentence_tokens = sent_tokenize(su_text)

[nltk_data] Error loading state_union: <urlopen error [Errno -2] Name
[nltk_data]     or service not known>


In [110]:
su_text[:500]

"PRESIDENT GEORGE W. BUSH'S ADDRESS BEFORE A JOINT SESSION OF THE CONGRESS ON THE STATE OF THE UNION\n \nJanuary 31, 2006\n\nTHE PRESIDENT: Thank you all. Mr. Speaker, Vice President Cheney, members of Congress, members of the Supreme Court and diplomatic corps, distinguished guests, and fellow citizens: Today our nation lost a beloved, graceful, courageous woman who called America to its founding ideals and carried on a noble dream. Tonight we are comforted by the hope of a glad reunion with the hus"

In [111]:
sentence_tokens[:5]

["PRESIDENT GEORGE W. BUSH'S ADDRESS BEFORE A JOINT SESSION OF THE CONGRESS ON THE STATE OF THE UNION\n \nJanuary 31, 2006\n\nTHE PRESIDENT: Thank you all.",
 'Mr. Speaker, Vice President Cheney, members of Congress, members of the Supreme Court and diplomatic corps, distinguished guests, and fellow citizens: Today our nation lost a beloved, graceful, courageous woman who called America to its founding ideals and carried on a noble dream.',
 'Tonight we are comforted by the hope of a glad reunion with the husband who was taken so long ago, and we are grateful for the good life of Coretta Scott King.',
 '(Applause.)',
 'President George W. Bush reacts to applause during his State of the Union Address at the Capitol, Tuesday, Jan. 31, 2006.']

In [112]:
# Look at POS for 5 first sentences of State of the Union address from 2006 from  past President George W. Bush.
try:
    for i in sentence_tokens[1:5]:
        words = nltk.word_tokenize(i)
        tagged = nltk.pos_tag(words)
        print(tagged)
except Exception as e:
    print(str(e))


[('Mr.', 'NNP'), ('Speaker', 'NNP'), (',', ','), ('Vice', 'NNP'), ('President', 'NNP'), ('Cheney', 'NNP'), (',', ','), ('members', 'NNS'), ('of', 'IN'), ('Congress', 'NNP'), (',', ','), ('members', 'NNS'), ('of', 'IN'), ('the', 'DT'), ('Supreme', 'NNP'), ('Court', 'NNP'), ('and', 'CC'), ('diplomatic', 'JJ'), ('corps', 'NN'), (',', ','), ('distinguished', 'JJ'), ('guests', 'NNS'), (',', ','), ('and', 'CC'), ('fellow', 'JJ'), ('citizens', 'NNS'), (':', ':'), ('Today', 'VB'), ('our', 'PRP$'), ('nation', 'NN'), ('lost', 'VBD'), ('a', 'DT'), ('beloved', 'VBN'), (',', ','), ('graceful', 'JJ'), (',', ','), ('courageous', 'JJ'), ('woman', 'NN'), ('who', 'WP'), ('called', 'VBD'), ('America', 'NNP'), ('to', 'TO'), ('its', 'PRP$'), ('founding', 'NN'), ('ideals', 'NNS'), ('and', 'CC'), ('carried', 'VBD'), ('on', 'IN'), ('a', 'DT'), ('noble', 'JJ'), ('dream', 'NN'), ('.', '.')]
[('Tonight', 'NN'), ('we', 'PRP'), ('are', 'VBP'), ('comforted', 'VBN'), ('by', 'IN'), ('the', 'DT'), ('hope', 'NN'), ('of

### Lemmatization with context

#### Penn TreeBank tags to WordNet pos

In [113]:
from nltk.corpus import wordnet
print(wordnet._FILEMAP)

{'a': 'adj', 'r': 'adv', 'n': 'noun', 'v': 'verb'}


In [114]:
# a simple converter 
def get_wordnet_pos(treebank_tag):
    """
    return WORDNET POS compliance to WORDNET lemmatization (a,n,r,v) 
    """
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        # As default pos in lemmatization is Noun
        return wordnet.NOUN

#### Contextual lemmatization with WordNetLemmatizer

In [115]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
try:
    for i in sentence_tokens[1:2]:
        words = nltk.word_tokenize(i)
        tagged = nltk.pos_tag(words)        
        lemma_pos_token = [(lemmatizer.lemmatize(w, pos=get_wordnet_pos(pos_tag)),pos_tag) for (w, pos_tag) in tagged] 
        print(lemma_pos_token)
except Exception as e:
    print(str(e))

[('Mr.', 'NNP'), ('Speaker', 'NNP'), (',', ','), ('Vice', 'NNP'), ('President', 'NNP'), ('Cheney', 'NNP'), (',', ','), ('member', 'NNS'), ('of', 'IN'), ('Congress', 'NNP'), (',', ','), ('member', 'NNS'), ('of', 'IN'), ('the', 'DT'), ('Supreme', 'NNP'), ('Court', 'NNP'), ('and', 'CC'), ('diplomatic', 'JJ'), ('corp', 'NN'), (',', ','), ('distinguished', 'JJ'), ('guest', 'NNS'), (',', ','), ('and', 'CC'), ('fellow', 'JJ'), ('citizen', 'NNS'), (':', ':'), ('Today', 'VB'), ('our', 'PRP$'), ('nation', 'NN'), ('lose', 'VBD'), ('a', 'DT'), ('beloved', 'VBN'), (',', ','), ('graceful', 'JJ'), (',', ','), ('courageous', 'JJ'), ('woman', 'NN'), ('who', 'WP'), ('call', 'VBD'), ('America', 'NNP'), ('to', 'TO'), ('it', 'PRP$'), ('founding', 'NN'), ('ideal', 'NNS'), ('and', 'CC'), ('carry', 'VBD'), ('on', 'IN'), ('a', 'DT'), ('noble', 'JJ'), ('dream', 'NN'), ('.', '.')]


### Application of NLP preprocessing techniques for SMS spam detection...

In [116]:
# SMS language is quite special, but you can try yet some NLP preprocessing techniques, and evaluate their interest,
# on the SMS spam detection problem...

# TODO: your code here

## Deepenings

- Sequences can be handled by SVM without transformation into fixed-sized vectors features by using the kernel trick and "string kernels"...

- __New trend__: vector representation of words (word2vec, ...)   
Not yet in scikit-learn, but you can look at http://zablo.net/blog/post/twitter-sentiment-analysis-python-scikit-word2vec-nltk-xgboost for an example using word2vec with gensim and scikit-learn...